# [Counting reads](https://htseq.readthedocs.io/en/release_0.11.1/counting.html)

A very typical use case for the `HTSeq` **library** is to for a given **list of genomic features** (such as `genes`, `exons`, `ChIP-Seq peaks`, or the like), how many sequencing **reads overlap each of the features**.

As a more complex example for using *HTSeq*, we supply the **script** `htseq-count`, which takes a `GTF` file with gene models and a `SAM` file and **counts** for each gene how many **reads map to it**; see Section [Counting reads in features with htseq-count](https://htseq.readthedocs.io/en/release_0.11.1/count.html#count).

The `htseq-count` **script**, however, has implementation details which were chosen with a specific use case in mind, namely to quantify gene expression for subsequent testing for differential expression, which is why, for example, the script does not count reads that map to multiple genes. For other applications, different resolutions of such ambiguities might be desirable, and then, a bioinformatician **may want to create her own counting script**. 

In the following, we expand on the coverage of this topic in the *Tour* ([A tour through HTSeq](https://htseq.readthedocs.io/en/release_0.11.1/tour.html#tour) and give building blocks which should make it possible to write such scripts also for bioinformaticians with only modest knowledge of Python.

## Preparing the feature array

Our general approach is to define a [GenomicArrayOfSets](https://htseq.readthedocs.io/en/release_0.11.1/genomic.html#HTSeq.GenomicArrayOfSets) and fill it with all the features we would like to get counts for.

Similar to the code shown in the *Tour*, we prepare such an object from the `GTF` file for yeast as follows:

In [3]:
import HTSeq

# select gtf file: instantiate GFF_Reader Object
gtf_file = HTSeq.GFF_Reader("data/Saccharomyces_cerevisiae.SGD1.01.56.gtf.gz")

# instantiate `GenomicArrayOfSets` for the `exons` features:
exons = HTSeq.GenomicArrayOfSets("auto", stranded=True)

## loop over all features in gtf file
for feature in gtf_file:
    
    ## store all exons in our `GenomicArrayOfSets`
    if feature.type == "exon":
        exons[feature.iv] += feature.attr["gene_id"]

A few things might be noteworthy here: For each **exon**, we just store the `gene ID` in the genomic array. Hence, **all exons** from the **same gene** are represented with the **same string**. This is deliberate, as we want to **count on the level of genes**, not exons, but could be done differently: storing the whole `feature` **object** in the `GenomicArrayOfSets` uses up noticeably more memory but allows to access more information in downstream processing.

Also note that in a `GTF` **file**, an **exon** that appears in **several transcripts** appear once for each transcripts. Because all these exons are represented by the same name, they will be collapsed to a **single value** in the `GenomicArrayOfSets`.

`GTF` **files** are not the only source of **feature annotations**. One could, as well, read a `BED` **file** or other text file with genomic coordinates of, say, *ChIP-Seq peaks*, putative enhancers, or any other kind of data. For example, if we have a **tab-separated text file** with **feature coordinates** in four columns – `feature ID`, `chromosome`, `start`, `end` – we might use:

In [4]:
# instantiate `GenomicArrayOfSets` for the `features`:
features =  HTSeq.GenomicArrayOfSets( "auto", stranded=False )

## loop over all features in myfeatures.txt
for line in open("data/myfeatures.txt" ):
    
    ## parse 'tab' delimited line
    fields = line.split( "\t" )
    
    ## `chromosome`, `start`, `end`
    iv = HTSeq.GenomicInterval(fields[1], int(fields[2]), int(fields[3]) )
    
    ## `feature ID`,
    features[ iv ] += fields[0]

FileNotFoundError: [Errno 2] No such file or directory: 'data/myfeatures.txt'

Here, we have assumed that the coordinates follow Python conventions: The first base of a chromosome is numbered 0, not 1, and the end position is not included in the interval. Remember to subtract or add 1 as necessary if this is not the case with your input data.

## Counting ungapped single-end reads

We start with the easiest case, that of **ungapped single-end reads**. We first recapitulate points already shown in the *Tour* and then add further refinements in the following.

If we have a `SAM` **file** with unmapped reads, we might use the following code:

In [37]:
import collections
counts = collections.Counter( )

#sam_file = HTSeq.SAM_Reader("my_alignments.sam")
sam_file = HTSeq.SAM_Reader("data/yeast_RNASeq_excerpt.sam")

for aln in sam_file:
    
    if not aln.aligned:
        counts["_unmapped"] += 1
        ## skips to next iteration
        continue
        
    gene_ids = set()
    for iv, val in features[ aln.iv ].steps():
        gene_ids |= val
    
    ## count
    if len(gene_ids) == 1:
        gene_id = list(gene_ids)[0]
        counts[gene_id] += 1
    elif len(gene_ids) == 0:
        counts["_no_feature"] += 1
    else:
        counts["_ambiguous"] += 1

for gene_id in counts:
    print(gene_id, counts[gene_id])

_no_feature 23663
_unmapped 1336


For the benefit of readers with only limited Python knowledge, we go through this code chunk step by step:

* The **variable** `counts` contains a **dictionary**, which will associate (keys) **gene IDs** with (values) **read counts**. We use a variant of Python’s usual `dict` **type**, namely the `Counter` **class** from the `collections` **module** in the standard library (from Python 2.7 onwards), which initialized any new key with the value zero. (Users of Python 2.6 can use collections.defaultdict(int) instead.)

* We then instantiate a `SAM_Reader` **object** (If you have a `BAM` **file**, use `BAM_Reader` instead) and run through all its record in a for loop. As described in the *Tour*, each record in the `SAM` **file** is provided to the loop body in the **variable** `aln`.

* We first check whether the **read might be unaligned**, and if so, increment a special counter that we call `_unmapped` (with an underscore as prefix to distinguish it from `gene IDs`).

* For the **aligned reads**, the **alignment’s genomic interval**, `aln.iv`, shows us the **interval covered by the read**. Using this as **an index** to `feature` gives us a view on this stretch of the `feature` container, in which we had stored the **exons**. The **iterator** `features[ almnt.iv ].steps()` returns pairs of the form `(iv, val)`, where `iv` is a **genomic interval** and `val` is the **set of gene IDs** associated with the **exons overlapping this step**. Using the `|=` **operator**, we get the **union of the sets** of all the steps in the initially empty set `gene_ids`, which, at the end, contains the **gene IDs** of **all genes that the read overlaps**. Remember that a `set` can contain each element at most once. Hence, even if we see the same gene in several steps (for example because the read overlaps with several exons), we still get it only once in `gene_ids`.

* We then treat three possible cases, namely that the **set** `gene_ids`:
    * contains **exactly one element**
    * it is **empty**,
    * contains **more than one element**.
 
The **first case** is the desired one: The **read overlaps** with **precisely one gene**, and we hence increase the count for this gene by one. Note that we need the idiom `list(gene_ids)[0]` to extract the name of this single gene from the set.

If the **read did not overlap with a gene** (len(gene_ids) == 0), we increase a special counter that we call `_no_feature`.

What should we do if the **read overlaps more than one gene**? Here, one might now come up with sophisticated logic to decide which gene to count the read for. To keep things simple, however, we simply count the read for none of the overlapped genes and instead increase the special counter `_ambiguous`.

* In the final two lines, we loop through the counter to print out the counts.

## Counting gapped single-end reads

### CIGAR Operations

The above code can be used as is e.g. for **ChIP-Seq data**, but for **RNA-Seq data**, we need an additional ingredient

When sequencing RNA, many reads will pass over an **exon-exon junction** and hence **align to two (or more) disjunct intervals** on the genome, tyically with an intron in between. If the reads have been aligned with a **splice-aware alignment tool** (e.g. STAR), such gapped alignment is indicated in the `SAM` **file** by the `CIGAR` (Compact Idiosyncratic Gapped Alignment Report) **string**.

*HTSeq* parses the `CIGAR` **string** and presents it in the `cigar` slot of a **class** `SAM_Alignment` **object** as a **list** of **class** `CigarOperation` **objects**.

As an example, consider a `SAM` **alignment record** describing a read that has been aligned to **position** `1000` on the `‘+’` **strand** of **chromosome** `chr1`, with **CIGAR string** `20M300N30M2I8M`. `CIGAR` strings have a number of operators:
```
M	Match 				Exact match of x positions
N	Alignment gap 		Next x positions on ref don’t match
D	Deletion  			Next x positions on ref don’t match
I	Insertion 			Next x positions on query don’t match
```
Following the **SAM specification** (please read it first if you are unfamiliar with CIGAR strings), this means an alignment as depicted here:

![img_cigar](https://htseq.readthedocs.io/en/release_0.11.1/_images/cigar.png)

The `SAM_Alignment` **object** will hence have in its `cigar` slot a list of 5 objects, each giving the information of one row of the table.

Note how some operations are associated with zero-length intervals on either the **reference** (i.e., `chromosome`) or the **query** (i.e., `read`). For example, the **intron** (`N200`) spans `200 bp` on the **chromosome** (`1020-1320`) but a zero-length interval on the **read** (`20-20`). In this manner, the `CigarOperation` **class** conveniently shows which intervals are affected by which operation.

### Counting with gapped reads

In the code above, we used this `for` loop

In [25]:
gene_ids = set()
for iv, val in features[ aln.iv ].steps():
    gene_ids |= val

to collect the **gene IDs** of all exons overlapped by the **reads interval**. `For` loop runs over the whole **interval covered** by the **aligned read**, i.e., in the figure above, it would run from **position** `1000` to **position** `1378` on **chromosome** `1`, including the **alignment gap** from `1020 to 1320`. By looking at each **cigar operation** separately we can correctly skip the gaps.

We only need to replace the `for` loop with the following **double** loop:

In [23]:
gene_ids = set()
for cig_op in aln.cigar:
    ## different from 'matched'
    if cig_op.type != "M":
        continue
    for iv, val in features[ cig_op.ref_iv ].steps():
        gene_ids |= val

In [35]:
#[cig_op.type for cig_op in aln.cigar]

The outer loop goes through the `CIGAR` **operation**, skipping all but the `match` (`"M"`) **operations**, and the inner loop inspects the **steps covered** by the `match` **operations** and collects the *gene_ids* in the `gene_ids` set variable. The rest of the code stays as above.

Of course, custom logic can be implemented her to infer useful information from other cigar operation types, but for the simple counting task at hand here, we do not need this.

### Dealing with multiple alignments

If the aligner finds **multiple alignments** for a **read**, these are typically reported in **multiple SAM records**. If the `SAM` **file** unsorted or sorted by alignment position, it is hard to look at all the possible alignments of a given read together, because the records with the alignments for a given read are spread throughout the file. If the purpose of the counting is subsequent testing for differential expression, it is often safest, anyway, to skip all multiply aligned reads (because a read that is counted for several genes may cause spurious calls of differential expression), and then, we merely need to recognize that a read has multiple alignments.

In the `htseq-count` **script**, this is done by two means: 
* First, many (but not all aligners) use the **optional field** `“NH”` (**number of hits**), which indicates the number of reported alignments. Testing for `aln.optional_field("NH") > 1` allows to find these read.

* Also, if there are **multiple good alignments**, without one of them being considered by the aligner to be more likely than the others, then the **alignment quality** (also denoted **mapping quality**, `MAPQ`, in the `SAM` specification) should be 3 or less. Hence, if one skips all reads with an alignment quality below, say, 10 (`aln.aQual < 10`), one will skip over all multiply aligned reads (provided the aligner indicates the mapping quality correctly, which is not always the case).

For more advanced use cases, it may be desirable to inspect **all reported alignment**, for example, to the chose one using some custom logic, or to aggregate information over all of them. If the `SAM` or `BAM` **file** has been sorted by read name then alternative alignments for the same read will be in adjacent lines or records. To facilitate handling this case, *HTSeq* offers the **function** `bundle_multiple_alignments`. It takes an iterator over `SAM_Alignment` **objects** (e.g., a `SAM_Reader` or `BAM_Reader` object) and returns an iterator over **lists** of `SAM_Alignment` **objects**. Each list contains only records describing **alignments for the same read**. For this to work, the `SAM` **file** has to be **sorted by read name** to ensure that mutiple alignments for the same read appear in **adjacent records**.

## Handling paired-end reads

<font color='red'> To be continued ... </font>